# MCP example with EXA MCP Server integrated

In [1]:
import os

import chromadb
import dotenv
from agents import Agent, Runner, function_tool, trace
from agents.mcp import MCPServerStreamableHttp

dotenv.load_dotenv()

True

Establecemos la conexión con nuestro Vector Store:

In [2]:
chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

In [ ]:
@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(query_texts=[query], n_results=max_results)

    if not results["documents"][0]:
        return f"No nutrition information found for: {query}"

    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results["documents"][0]):
        metadata = results["metadatas"][0][i]
        food_item = metadata["food_item"].title()
        calories = metadata["calories_per_100g"]
        category = metadata["food_category"].title()

        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )

    return "Nutrition Information:\n" + "\n".join(formatted_results)

### Creamos la conexion con el servidor MCP de EXA

Aqui definimos el servidor MCP 'Exa Search' agregando algunos parametros para que funcione correctamente.

> La clase `MCPServerStreamableHttp` sirve para conectar un agente con servicios externos mediante el protocolo MCP (Model Context Protocol) a través de HTTP streaming.

Es un puente de comunicación que permite que tu agente acceda a herramientas y capacidades externas que no están disponibles localmente:

- Conecta con APIs externas mediante HTTP
- Expone las herramientas del servicio externo al agente
- Maneja la comunicación bidireccional entre tu agente y el servicio


In [13]:
# Argumentos:
# name - nombramos el servidor MCP
# usamos 'params' para pasarle otros parametros:
#       API Key de EXA Search
#       Vamos a definir el timeout en 30 segundos
#       Por si tarda mucho en la conexión ponemos un timeout de la sesión del cliente en 30 segundos
#       Activamos el caching de la lista de tools para que no se tenga que volver a cargar la lista de tools en cada ejecución
#       Ponemos un maximo de 1 intento de retry   

exa_search_mcp = MCPServerStreamableHttp(
    name="Exa Search MCP",
    params={"url":f"https://mcp.exa.ai/mcp?exaApiKey={os.getenv('EXA_API_KEY')}",
    "timeout":30,
    },
    client_session_timeout_seconds=30,
    cache_tools_list=True,
    max_retry_attempts=1,
    )
        

In [ ]:
# primero vamos a conectar el MCP Server
await exa_search_mcp.connect()

### Creacion del Agente

Vamos a crear un agente que use la 'tool' que habiamos creado antes y elque tambien use el server MCP de 'EXA Search'

In [ ]:
# Definimos el Agente (que hemos importado de la clase 'agents')
calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:
        1) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        2) If you couldn't find the exact match for the food or you need to look up the ingredients, search the EXA web to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie
        information of the ingredients to make sure the information you provide is consistent.
        3) Then, if necessary, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use Exa Search to find the exact recipe and ingredients.
    * Once you know the ingredients, use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times.
    """,
    tools=[calorie_lookup_tool],
    mcp_servers=[exa_search_mcp],
)

> Esta query no deberia usar el ExaSearch:

In [19]:
with trace("Nutrition Assistant with MCP - Only uses calorie_lookup_tool"):
    result = await Runner.run(
        calorie_agent_with_search,
        "How many calories are in total in a banana and an apple? Also give calories per 100g",
    )
    print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    - Banana (medium, ~118 g): ~105 kcal
    - Apple (medium, ~182 g): ~95 kcal
    - Total (banana + apple): ~200 kcal
    
    Calories per 100 g:
    - Banana: 89 kcal/100 g
    - Apple: 52 kcal/100 g
- 7 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [20]:
with trace("Nutrition Assistant with MCP "):
    result = await Runner.run(
        calorie_agent_with_search, "How many calories are in an english breakfast?"
    )
    print(result.final_output)

A typical full English breakfast is about 1,600 kcal per serving (ranges ~1,000–1,600 kcal depending on ingredients).

Common ingredients (per 1 serving, as referenced by recipe sources):
- Fried egg (60 g): 90 kcal
- Pork sausages (110 g): 330 kcal
- Bacon rashers (70 g): 250 kcal
- Baked beans in tomato sauce (150 g): 120 kcal
- Hash browns (100 g): 280 kcal
- Fried bread (80 g): 280 kcal
- Grilled tomato (80 g): 15 kcal
- Black pudding (60 g): 180 kcal
- Butter (10 g): 75 kcal
- Oil (for frying, ~2 tsp): varies (included in overall, ~20–40 kcal)

Total: about 1,620 kcal per serving (adjust up or down with portion sizes). Let me know if you want a version based on different portions.
